# Construindo uma Arquitetura RAG Avançada


In [ ]:
# Instalação das bibliotecas necessárias para o notebook

# Biblioteca para usar a API da Groq (modelos de linguagem otimizados para baixa latência)
!pip install -q groq

# Biblioteca de embeddings pré-treinados para transformar textos em vetores (usada em RAG)
!pip install -q sentence-transformers

# Biblioteca para acessar conteúdos da Wikipedia de forma programática
!pip install -q wikipedia-api

# Conjunto de bibliotecas do LangChain e ecossistema:
# - langchain-nvidia-ai-endpoints: integração com os modelos NIM da NVIDIA
# - langchain-community: integrações da comunidade com loaders, embeddings, etc.
# - langchain: framework principal para RAG, agentes e pipelines
# - langgraph: orquestração de fluxos com grafos de estados
# - tavily-python: integração com o motor de busca Tavily
# - beautifulsoup4 e lxml: usados para parsing de HTML quando carregamos páginas da web
!pip install -qU langchain-nvidia-ai-endpoints langchain-community langchain langgraph tavily-python beautifulsoup4 lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
  

In [ ]:
# Bibliotecas padrão do Python
import json   # para manipulação de dados no formato JSON
import os     # para acesso a variáveis de ambiente e operações do sistema

# Bibliotecas numéricas e utilitárias
import numpy as np        # cálculos numéricos e vetoriais
import operator           # funções utilitárias para operações (ex: itemgetter)

# Google Colab
from google.colab import userdata   # para acessar dados/segredos do usuário armazenados no Colab

# APIs e Modelos
from google import genai            # integração com modelos generativos do Google (Gemini, etc.)
from groq import Groq               # cliente para usar modelos da Groq (otimização de LLMs)
from openai import OpenAI           # cliente oficial para API da OpenAI

# Visualização
from IPython.display import Image, display   # exibir imagens diretamente no notebook

# LangChain - manipulação de texto, dados e fluxos
from langchain.text_splitter import RecursiveCharacterTextSplitter  # dividir textos longos em chunks
from langchain_community.document_loaders import WebBaseLoader      # carregar dados de páginas da web
from langchain_community.tools.tavily_search import TavilySearchResults  # integração com motor Tavily
from langchain_core.vectorstores import InMemoryVectorStore         # armazenamento vetorial em memória
from langchain_core.messages import HumanMessage, SystemMessage     # tipos de mensagens usadas em LLMs
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA  # embeddings e chat da NVIDIA NIM
from langchain.schema import Document                               # estrutura padrão de documentos no LangChain

# LangGraph - orquestração de fluxos de agentes
from langgraph.graph import END, StateGraph  # definição de estados e término do grafo

# Validação de dados e modelos
from pydantic import BaseModel, Field        # validação e definição de schemas de dados

# Embeddings
from sentence_transformers import SentenceTransformer  # biblioteca de embeddings da Hugging Face

# Tipagem avançada
from typing_extensions import TypedDict, List, Annotated, Literal  # anotações de tipos mais sofisticadas

# Acesso à Wikipedia
from wikipediaapi import Wikipedia   # biblioteca para buscar conteúdos da Wikipedia

### Como conseguir as API keys?

Busque no Google "Nome da empresa API key", funciona em 90% das vezes. Todas as API keys abaixo são de planos free.

In [ ]:
# Configuração de variáveis de ambiente para os serviços usados no notebook

# Ativa o sistema de tracing da LangChain para monitorar execução de fluxos e agentes
os.environ["LANGCHAIN_TRACING_V2"] = "True"

# Define o endpoint da LangSmith (plataforma de observabilidade da LangChain)
os.environ["LANGCHAIN_ENDPOINT"] = 'https://api.smith.langchain.com'

# Define a chave de API da LangSmith, recuperada dos segredos do Colab
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGSMITH_API_KEY")

# Nome do projeto no LangSmith para organizar logs e execuções
os.environ["LANGSMITH_PROJECT"] = "pr-crushing-nexus-98"

# Define a chave de API do Tavily (motor de busca inteligente), vinda dos segredos do Colab
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

# Define a chave de API da NVIDIA, também recuperada dos segredos do Colab
os.environ["NVIDIA_API_KEY"] = userdata.get("NVIDIA_API_KEY")

In [ ]:
# Exemplo de inicialização de diferentes clientes de LLM.
# As duas primeiras linhas estão comentadas, mas mostram como seria a conexão
# com outros provedores:
# - Gemini (Google) usando a chave GEMINI_API_KEY
# - OpenAI usando a chave OPENAI_API_KEY

# Inicialização do cliente Groq, utilizando a chave armazenada no Colab.
client = Groq(api_key=userdata.get("GROQ_API_KEY"))

# Definição do modelo a ser usado.
# Aqui está selecionado o LLaMA 3.1 de 70B parâmetros.
# A linha de baixo é uma alternativa para usar a versão de 8B parâmetros.
model_id = "meta/llama-3.1-70b-instruct"
# model_id = "meta/llama-3.1-8b-instruct"

# Criação da instância de ChatNVIDIA para chamadas de LLM,
# configurada com o modelo escolhido e temperatura 0 (respostas mais determinísticas).
llm = ChatNVIDIA(model=model_id, temperature=0)

In [ ]:
# URLs que servirão como base de conhecimento.
# Cada link contém artigos da Lilian Weng sobre agentes, prompt engineering e ataques adversariais em LLMs.
urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

# Carregamento dos documentos a partir das URLs usando o WebBaseLoader.
# Cada URL retorna uma lista de documentos, por isso é criada uma lista de listas.
docs = [WebBaseLoader(url).load() for url in urls]

# Flatten da lista de listas em uma única lista de documentos.
docs_list = [item for sublist in docs for item in sublist]

# Criação de um divisor de texto que utiliza o tokenizador do tiktoken.
# O texto será quebrado em pedaços de até 250 tokens, sem sobreposição entre eles.
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

# Criação de uma base vetorial em memória a partir dos documentos divididos.
# É utilizado o modelo NVIDIA para gerar embeddings (NV-Embed-QA).
vectorstore = InMemoryVectorStore.from_documents(
    documents=doc_splits,
    embedding=NVIDIAEmbeddings(model='NV-Embed-QA'),
)

# Transformação da base vetorial em um "retriever".
# O retriever permite buscar os 3 documentos mais relevantes para uma consulta.
retriever = vectorstore.as_retriever(k=3)

In [ ]:
# Definição do schema de saída para o roteador de consultas.
# A classe BaseModel (do Pydantic) define que toda consulta deve ser roteada
# para uma das duas opções: "vectorstore" ou "websearch".
class RouteQuery(BaseModel):
    """Direcione uma consulta para a fonte de informação mais útil."""

    # O campo datasource só pode assumir os valores "vectorstore" ou "websearch".
    datasource: Literal["vectorstore", "websearch"] = Field(
        ...,
        description=(
            "Dada a pergunta do usuário, escolha encaminhá-la para "
            "uma pesquisa na web ou para uma vector database."
        ),
    )

# Configuração do LLM para retornar saídas estruturadas no formato definido pelo Pydantic.
# Isso garante que a resposta sempre siga o schema da classe RouteQuery.
structured_llm_router = llm.with_structured_output(RouteQuery)

# Prompt do sistema que orienta o modelo a tomar a decisão correta.
# Ele explica quando usar a base vetorial (vectorstore) e quando usar a busca na web.
router_instructions = """Você é especialista em encaminhar a pergunta de um usuário para um vectorstore ou busca na web.
O vectorstore contém documentos relacionados a agentes, engenharia de prompts e ataques adversários.
Use o vectorstore para perguntas sobre esses tópicos. Para todos os outros, e especialmente para eventos atuais, use a busca na web.
Retorna uma saída estruturada com uma única chave, fonte de dados, que pode ser "websearch" ou "vectorstore", dependendo da pergunta."""

# Exemplos de testes do roteador:

# Exemplo 1: Pergunta sobre campeonato brasileiro → deve usar busca na web
test_web_search = structured_llm_router.invoke(
    [SystemMessage(content=router_instructions)]
    + [
        HumanMessage(
            content="Quem é o favorito para vencer o campeonato brasileiro de 2025?"
        )
    ]
)

# Exemplo 2: Pergunta sobre lançamento recente de LLMs → também deve usar busca na web
test_web_search_2 = structured_llm_router.invoke(
    [SystemMessage(content=router_instructions)]
    + [HumanMessage(content="Teve algum modelo LLM lançado hoje?")]
)

# Exemplo 3: Pergunta sobre memórias de agentes → deve usar vectorstore
test_vector_store = structured_llm_router.invoke(
    [SystemMessage(content=router_instructions)]
    + [HumanMessage(content="Quais são os tipos de memórias de agentes?")]
)

# Exibe os resultados de cada teste de roteamento
print(
    test_web_search,
    test_web_search_2,
    test_vector_store,
)

datasource='websearch' datasource='websearch' datasource='vectorstore'


## Avaliador de Retrieval

In [ ]:
# Definição de um modelo de dados Pydantic para padronizar a resposta do avaliador de relevância.
# Essa classe especifica que a resposta deve conter apenas uma chave: binary_score,
# cujo valor deve ser "yes" ou "no".
class GradeDocuments(BaseModel):
    """Classificação binária para a verificação de relevância dos documentos recuperados.
    """

    # Campo obrigatório que define a classificação do documento
    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )

# Configuração do LLM para retornar saídas estruturadas no formato GradeDocuments.
# Isso garante que a resposta do modelo siga o schema definido.
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Instruções para o avaliador (prompt de sistema).
# O modelo deve atuar como um juiz que avalia a relevância de documentos recuperados
# em relação à pergunta do usuário.
doc_grader_instructions = """Você é um avaliador que avalia a relevância de um documento recuperado para a pergunta de um usuário.
Se o documento contiver palavras-chave ou significado semântico relacionado à pergunta, classifique-o como relevante."""

# Prompt de avaliação, formatado com placeholders para o documento e a pergunta.
# O modelo deve verificar se há pelo menos alguma relação entre eles.
doc_grader_prompt = """Aqui está o documento recuperado: \n\n {document} \n\n Aqui está a pergunta do usuário: \n\n {question}.

Avalie cuidadosa e objetivamente se o documento contém pelo menos alguma informação relevante para a pergunta.
Retorne uma saída estruturada com uma única chave, binary_score, que é uma classificação "yes" ou "no" para indicar se o documento
contém pelo menos alguma informação relevante para a pergunta
"""

# ----------------
# Testando o avaliador
# ----------------

# Pergunta de teste do usuário
question = "O que é prompt do tipo chain-of-thought?"

# Recupera documentos da base vetorial relacionados à pergunta
docs = retriever.invoke(question)

# Seleciona o conteúdo de um dos documentos recuperados
doc_txt = docs[1].page_content

# Substitui placeholders no prompt pelo documento e pela pergunta
doc_grader_prompt_formatted = doc_grader_prompt.format(
    document=doc_txt, question=question
)

# Invoca o avaliador (LLM) passando:
# - as instruções do avaliador (SystemMessage)
# - o prompt formatado com documento e pergunta (HumanMessage)
result = structured_llm_grader.invoke(
    [SystemMessage(content=doc_grader_instructions)]
    + [HumanMessage(content=doc_grader_prompt_formatted)]
)

# Exibe o resultado (esperado: {'binary_score': 'yes'} ou {'binary_score': 'no'})
result

GradeDocuments(binary_score='yes')

## Geração de resposta pelo LLM

In [ ]:
# Prompt contextualizado usando RAG
# O prompt instrui o LLM a usar apenas o contexto fornecido para responder,
# pensar cuidadosamente e ser conciso (máx. 3 frases).
rag_prompt = """Você é um assistente para tarefas de resposta a perguntas.
Aqui está o contexto a ser usado para responder à pergunta:
{context}

Pense cuidadosamente sobre o contexto acima.
Agora, revise a pergunta do usuário:
{question}

Responda a esta pergunta usando apenas o contexto acima.
Use no máximo três frases e seja conciso.

Resposta:"""

# Formatação da resposta
# Concatena o conteúdo textual (page_content) de cada Document em uma string única,
# separando documentos por duas quebras de linha para manter clareza.
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

# Testando
# Recupera os documentos mais relevantes usando o retriever (assume que `question` existe)
docs = retriever.invoke(question)

# Converte a lista de Document em texto para inserir no prompt
docs_txt = format_docs(docs)

# Substitui os placeholders do prompt com o contexto e a pergunta
rag_prompt_formatted = rag_prompt.format(context=docs_txt, question=question)

# Chama o LLM passando a mensagem do usuário contendo o prompt pronto
# (dependendo do wrapper, você pode também incluir um SystemMessage com instruções do sistema)
generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])

# Imprime o conteúdo gerado pelo modelo
print(generation.content)

Chain of thought (CoT) prompting is a technique that generates a sequence of short sentences to describe reasoning logics step by step, known as reasoning chains or rationales, to eventually lead to the final answer. This technique is particularly beneficial for complicated reasoning tasks, especially when using large models with more than 50B parameters. CoT prompting helps to decompose hard tasks into smaller and simpler steps, making it easier for the model to arrive at a solution.


## Avaliador de Alucinação

In [ ]:
# Define um modelo de dados Pydantic para classificar se uma resposta contém alucinações
# (informações não fundamentadas nos fatos fornecidos)
class GradeHallucinations(BaseModel):
    """Classificação binária para alucinação presente na resposta gerada."""

    # Campo obrigatório que indica se a resposta segue os fatos ('yes') ou contém alucinações ('no')
    binary_score: str = Field(
        description="A resposta está de acordo com os fatos, 'yes' ou 'no'"
    )

# Configura o LLM para retornar saídas estruturadas conforme o schema definido
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Instruções detalhadas para o avaliador (prompt de sistema)
# Ensina o LLM a atuar como um professor corrigindo se a resposta do "aluno" segue os fatos
hallucination_grader_instructions = """
Você é um professor corrigindo um teste. Você receberá os Fatos e uma Resposta do Estudante.

Aqui estão os critérios de avaliação a serem seguidos:

(1) Certifique-se de que a Resposta do Estudante esteja fundamentada nos Fatos.
(2) Certifique-se de que a Resposta do Estudante não contenha informações "alucinadas" fora do escopo dos Fatos.

Classificação:

Uma classificação "yes" significa que a resposta do aluno atende a todos os critérios. Esta é a classificação mais alta (melhor).
Uma classificação "no" significa que a resposta do aluno não atende a todos os critérios. Esta é a classificação mais baixa que você pode dar.
Explique seu raciocínio passo a passo para garantir que seu raciocínio e conclusão estejam corretos.
Evite simplesmente declarar a resposta correta logo no início."""

# Prompt de avaliação, formatado com placeholders para os documentos (fatos) e a geração do modelo
# O LLM deve retornar uma classificação binária indicando se a resposta está fundamentada nos fatos
hallucination_grader_prompt = """Fatos: \n\n {documents} \n\n Resposta do estudante: {generation}.
Retorne a saída estruturada com duas chaves, binary_score é uma classificação 'yes' ou 'no' para indicar se a Resposta do Estudante está baseada nos Fatos."""

# Substitui placeholders no prompt com os documentos reais e a resposta gerada pelo LLM
hallucination_grader_prompt_formatted = hallucination_grader_prompt.format(
    documents=docs_txt, generation=generation.content
)

# Invoca o LLM para avaliar se a resposta contém alucinações, passando:
# - instruções do avaliador (SystemMessage)
# - prompt formatado com fatos e resposta do estudante (HumanMessage)
result = structured_llm_grader.invoke(
    [SystemMessage(content=hallucination_grader_instructions)]
    + [HumanMessage(content=hallucination_grader_prompt_formatted)]
)

# Exibe o resultado da avaliação (esperado: {'binary_score': 'yes'} ou {'binary_score': 'no'})
result

GradeDocuments(binary_score='yes')

## Avaliador de Resposta Final

In [ ]:
# Define um modelo de dados Pydantic para classificar se a resposta do estudante responde à pergunta
class GradeAnswer(BaseModel):
    """Classificação binária para avaliar se a resposta corresponde à pergunta."""

    # Campo obrigatório que indica se a resposta atende à pergunta ('yes') ou não ('no')
    binary_score: str = Field(
        description="Resposta de fato responde a pergunta, 'yes' or 'no'"
    )

# Configura o LLM para retornar saídas estruturadas conforme o schema definido
structured_llm_grader = llm.with_structured_output(GradeAnswer)

# Instruções detalhadas para o avaliador (prompt de sistema)
# Ensina o LLM a atuar como um professor corrigindo se a resposta do aluno responde à pergunta
answer_grader_instructions = """Você é um professor corrigindo um teste. Você receberá uma PERGUNTA e uma RESPOSTA DO ESTUDANTE.

Aqui estão os critérios de avaliação a serem seguidos:

(1) A RESPOSTA DO ESTUDANTE ajuda a responder à PERGUNTA

Classificação:
Uma classificação "yes" significa que a RESPOSTA DO ESTUDANTE atende a todos os critérios. Esta é a classificação mais alta (melhor).
O aluno pode receber uma classificação "yes" se a resposta contiver informações adicionais que não sejam explicitamente solicitadas na pergunta.
Uma classificação "no" significa que a RESPOSTA DO ESTUDANTE não atende a todos os critérios. Esta é a classificação mais baixa que você pode dar.
Explique seu raciocínio passo a passo para garantir que seu raciocínio e conclusão estejam corretos.
Evite simplesmente declarar a resposta correta logo no início."""

# Prompt de avaliação, formatado com placeholders para a pergunta e a resposta do estudante
# O LLM deve retornar uma saída binária indicando se a resposta atinge os critérios
answer_grader_prompt = """PERGUNTA: \n\n {question} \n\n RESPOSTA DO ESTUDANTE: {generation}.
Returne uma structured output com binary_score sendo 'yes' ou 'no' indicando se a RESPOSTA DO ESTUDANTE atinge os critérios."""

# Substitui placeholders no prompt com a pergunta real e a resposta fornecida
question = "Quais modelos da série Llama 3.2 foram lançados por último?"
answer = "Os modelos Llama 3.2 lançados por último incluem dois modelos de visão: Llama 3.2 11B Vision Instruct e Llama 3.2 90B Vision Instruct. Esses modelos fazem parte da primeira incursão da Meta em IA multimodal e rivalizam com modelos fechados como o Claude 3 Haiku da Anthropic e o GPT-4o mini da OpenAI em raciocínio visual. Eles substituem os antigos modelos Llama 3.1 de texto."

answer_grader_prompt_formatted = answer_grader_prompt.format(
    question=question, generation=answer
)

# Invoca o LLM para avaliar se a resposta corresponde à pergunta, passando:
# - instruções do avaliador (SystemMessage)
# - prompt formatado com pergunta e resposta (HumanMessage)
result = structured_llm_grader.invoke(
    [SystemMessage(content=answer_grader_instructions)]
    + [HumanMessage(content=answer_grader_prompt_formatted)]
)

# Exibe o resultado da avaliação (esperado: {'binary_score': 'yes'} ou {'binary_score': 'no'})
result

GradeAnswer(binary_score='yes')

In [ ]:
# Instanciação do mecanismo de busca Tavily com limite de 3 resultados
web_search_tool = TavilySearchResults(k=3)

# Definição do estado do grafo como um dicionário tipado (TypedDict)
class GraphState(TypedDict):
    """
    Representa o estado de cada nó no grafo, contendo informações
    que serão propagadas e modificadas durante o processamento.
    """

    question: str  # Pergunta do usuário
    generation: str  # Resposta gerada pelo LLM
    web_search: str  # Decisão binária para realizar busca na web
    max_retries: int  # Número máximo de tentativas para gerar uma resposta
    answers: int  # Contador de respostas geradas
    loop_step: Annotated[int, operator.add]  # Contador de etapas no loop
    documents: List[str]  # Lista de documentos recuperados

In [ ]:
### Nodes

# Nó responsável por recuperar documentos do vectorstore
def retrieve(state):
    """
    Recupera documentos relevantes da base vetorial com base na pergunta do usuário.
    Adiciona os documentos recuperados ao estado do grafo.
    """
    print("---RECUPERAR---")
    question = state["question"]
    documents = retriever.invoke(question)
    return {"documents": documents}


# Nó responsável por gerar uma resposta usando RAG
def generate(state):
    """
    Gera uma resposta do LLM baseada nos documentos recuperados.
    Atualiza o estado do grafo com a resposta gerada e incrementa o passo do loop.
    """
    print("---GERAR---")
    question = state["question"]
    documents = state["documents"]
    loop_step = state.get("loop_step", 0)

    docs_txt = format_docs(documents)
    rag_prompt_formatted = rag_prompt.format(context=docs_txt, question=question)
    generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
    return {"generation": generation, "loop_step": loop_step + 1}


# Nó que avalia a relevância dos documentos recuperados
def grade_documents(state):
    """
    Avalia cada documento recuperado para verificar se é relevante para a pergunta.
    Filtra documentos irrelevantes e define uma flag 'web_search' se algum documento não for relevante.
    """
    print("---CHECANDO RELEVÂNCIA DO DOCUMENTO PARA A QUESTÃO---")
    question = state["question"]
    documents = state["documents"]

    filtered_docs = []
    web_search = "No"
    structured_llm_grader = llm.with_structured_output(GradeDocuments)
    for d in documents:
        doc_grader_prompt_formatted = doc_grader_prompt.format(
            document=d.page_content, question=question
        )

        result = structured_llm_grader.invoke(
            [SystemMessage(content=doc_grader_instructions)]
            + [HumanMessage(content=doc_grader_prompt_formatted)]
        )
        grade = result.binary_score
        if grade.lower() == "yes":
            print("---AVALIAÇÃO: DOCUMENTO RELEVANTE---")
            filtered_docs.append(d)
        else:
            print("---AVALIAÇÃO: DOCUMENTO NÃO É RELEVANTE---")
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "web_search": web_search}


# Nó que realiza busca na web se necessário
def web_search(state):
    """
    Executa uma busca na web com base na pergunta do usuário.
    Adiciona os resultados da busca ao conjunto de documentos do estado.
    """
    print("---WEB SEARCH---")
    question = state["question"]
    documents = state.get("documents", [])

    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    documents.append(web_results)
    return {"documents": documents}


### Edges

# Nó que roteia a pergunta para RAG ou busca na web
def route_question(state):
    """
    Decide se a pergunta deve ser enviada para o vectorstore (RAG) ou para busca na web.
    Baseado na decisão do LLM de roteamento.
    """
    print("---ROUTE QUESTION---")
    structured_llm_router = llm.with_structured_output(RouteQuery)
    route_question = structured_llm_router.invoke(
        [SystemMessage(content=router_instructions)]
        + [HumanMessage(content=state["question"])]
    )
    source = route_question.datasource
    if source == "websearch":
        print("---ROTEAR QUESTÃO PARA BUSCA NA WEB---")
        return "websearch"
    elif source == "vectorstore":
        print("---ROTEAR QUESTÃO PARA O RAG---")
        return "vectorstore"


# Nó que decide se devemos gerar resposta ou realizar busca na web
def decide_to_generate(state):
    """
    Determina se a resposta deve ser gerada pelo LLM ou se é necessário buscar na web.
    Baseado na relevância dos documentos filtrados.
    """
    print("---OBSERVAÇÃO DE DOCUMENTOS AVALIADOS---")
    web_search_flag = state["web_search"]

    if web_search_flag == "Yes":
        print("---DECISÃO: NEM TODOS OS DOCUMENTOS SÃO RELEVANTES, INCLUIR PESQUISA NA WEB---")
        return "websearch"
    else:
        print("---DECISÃO: GERAR---")
        return "generate"


# Nó que avalia se a geração do LLM está baseada nos documentos e responde à pergunta
def grade_generation_v_documents_and_question(state):
    """
    Avalia se a resposta gerada pelo LLM:
    1. Está fundamentada nos documentos (sem alucinações)
    2. Responde efetivamente à pergunta do usuário

    Retorna qual deve ser o próximo nó a ser chamado:
    - 'useful' se a resposta estiver correta e baseada nos fatos
    - 'not useful' se não responder à pergunta
    - 'not supported' se não estiver baseada nos fatos
    - 'max retries' se o número máximo de tentativas foi atingido
    """
    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]
    max_retries = state.get("max_retries", 3)

    hallucination_grader_prompt_formatted = hallucination_grader_prompt.format(
        documents=format_docs(documents), generation=generation.content
    )
    structured_llm_grader = llm.with_structured_output(GradeHallucinations)
    result = structured_llm_grader.invoke(
        [SystemMessage(content=hallucination_grader_instructions)]
        + [HumanMessage(content=hallucination_grader_prompt_formatted)]
    )
    grade = result.binary_score

    if grade == "yes":
        print("---DECISÃO: GERAÇÃO ESTÁ BASEADA NOS FATOS---")
        print("---AVALIAÇÃO GERAÇÃO vs QUESTÃO---")
        answer_grader_prompt_formatted = answer_grader_prompt.format(
            question=question, generation=generation.content
        )
        structured_llm_grader = llm.with_structured_output(GradeAnswer)
        result = structured_llm_grader.invoke(
            [SystemMessage(content=answer_grader_instructions)]
            + [HumanMessage(content=answer_grader_prompt_formatted)]
        )
        grade = result.binary_score
        if grade == "yes":
            print("---DECISÃO: GERAÇÃO RESPONDE A PERGUNTA---")
            return "useful"
        elif state["loop_step"] <= max_retries:
            print("---DECISÃO: GERAÇÃO NÃO RESPONDE A PERGUNTA---")
            return "not useful"
        else:
            print("---DECISÃO: MÁXIMO DE TENTATIVAS ATINGIDO---")
            return "max retries"
    elif state["loop_step"] <= max_retries:
        print("---DECISÃO: GERAÇÃO NÃO ESTÁ BASEADA NOS FATOS, TENTAR NOVAMENTE---")
        return "not supported"
    else:
        print("---DECISÃO: MÁXIMO DE TENTATIVAS ATINGIDO---")
        return "max retries"

In [ ]:
# Criação do grafo de estados usando a classe StateGraph
workflow = StateGraph(GraphState)

# Adiciona os nós do grafo com suas respectivas funções
workflow.add_node("websearch", web_search)            # Nó para busca na web
workflow.add_node("retrieve", retrieve)              # Nó para recuperar documentos do vectorstore
workflow.add_node("grade_documents", grade_documents)  # Nó para avaliar relevância dos documentos
workflow.add_node("generate", generate)              # Nó para gerar resposta usando RAG

# Define o ponto de entrada condicional, roteando a pergunta para web ou vectorstore
workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": "websearch",      # Se roteador decidir websearch, entra no nó 'websearch'
        "vectorstore": "retrieve",     # Se roteador decidir vectorstore, entra no nó 'retrieve'
    },
)

# Define as transições simples entre os nós
workflow.add_edge("websearch", "generate")  # Após buscar na web, gera resposta
workflow.add_edge("retrieve", "grade_documents")  # Após recuperar documentos, avalia relevância

# Define transições condicionais após avaliação dos documentos
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",  # Se documentos irrelevantes, fazer busca na web
        "generate": "generate",    # Se documentos relevantes, gerar resposta
    },
)

# Define transições condicionais após geração da resposta
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",  # Geração não baseada em fatos, tentar novamente
        "useful": END,                 # Resposta útil, fim do fluxo
        "not useful": "websearch",     # Resposta não útil, realizar busca na web
        "max retries": END,            # Máximo de tentativas atingido, fim do fluxo
    },
)

# Compila o grafo e exibe visualmente usando Mermaid
graph = workflow.compile()
display(Image(graph.get_graph().draw_mermaid_png()))


In [ ]:
# Define a entrada inicial para o grafo de workflow
inputs = {
    "question": "Quais são os tipos de memória de agentes?",  # Pergunta do usuário
    "max_retries": 3  # Número máximo de tentativas para gerar uma resposta válida
}

# Executa o grafo de forma interativa (streaming), exibindo os eventos à medida que ocorrem
for event in graph.stream(inputs, stream_mode="values"):
    print(event)  # Imprime cada atualização do estado do grafo durante a execução

---ROUTE QUESTION---
---ROUTE QUESTION TO RAG---
{'question': 'What are the types of agent memory?', 'max_retries': 3, 'loop_step': 0}
---RETRIEVE---
{'question': 'What are the types of agent memory?', 'max_retries': 3, 'loop_step': 0, 'documents': [Document(id='3d406e2f-0927-42a9-bee2-7e811ffab2fd', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/', 'title': "LLM Powered Autonomous Agents | Lil'Log", 'description': 'Building agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks do

In [ ]:
# Define a entrada inicial para o grafo de workflow
inputs = {
    "question": "Qual é o modelo mais recente da série Llama da Meta?",  # Pergunta do usuário
    "max_retries": 3,  # Número máximo de tentativas para gerar uma resposta válida
}

# Executa o grafo de forma interativa (streaming), exibindo os eventos à medida que ocorrem
for event in graph.stream(inputs, stream_mode="values"):
    print(event)  # Imprime cada atualização do estado do grafo durante a execução

---ROUTE QUESTION---
---ROUTE QUESTION TO WEB SEARCH---
{'question': 'What are the most recent llama models released?', 'max_retries': 3, 'loop_step': 0}
---WEB SEARCH---
{'question': 'What are the most recent llama models released?', 'max_retries': 3, 'loop_step': 0, 'documents': [Document(metadata={}, page_content='Wikipedia\n\n# Llama (language model)\n\nLlama (Large Language Model Meta AI) is a family of large language models (LLMs) released by Meta AI starting in February 2023. The latest version is Llama 4, released in April 2025. [...] ## Llama 2\n\nOn July 18, 2023, in partnership with Microsoft, Meta announced Llama 2 (stylized as LLaMa 2), the next generation of Llama. Meta trained and released Llama 2 in three model sizes: 7, 13, and 70 billion parameters. The model architecture remains largely unchanged from that of Llama 1 models, but 40% more data was used to train the foundational models. [...] On April 18, 2024, Meta released Llama 3 with two sizes: 8B and 70B parameter